<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/TestingModelos/YoloV8Nano_100Epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de detección de espermatozoides usando YoloV8 Nano para iniciar

Se realiza la instalación de las librerías necesarias para el procedimiento:

In [1]:
%%capture
!pip install ultralytics

In [2]:
%%capture
!pip install OpenCV
!pip install Pillow

In [3]:
%%capture
!pip install opencv-contrib-python-headless

Cargamos el DataSet y un video de prueba para realizar la detección:

In [4]:
%%capture
# Carga de DataSet
!wget https://www.dropbox.com/scl/fi/8bjlyvmulkwb4l09lgbzc/dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q&st=7hkm6ksd&dl=0
!wget https://www.dropbox.com/scl/fi/f0wmb3677x28qwt3np0t8/acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc&st=17daydxb&dl=1


In [5]:
%%capture
!mv dataSet.zip?rlkey=dmqb2t13gv9ywi4g9bsesdq6q dataSet.zip
!unzip dataSet.zip

In [6]:
%%capture
!mv acA1920-155um__22949301__20210628_094625267.avi?rlkey=2lfsr3mspsh5potd64pkb7coc test.avi

Cargamos un modelo preentrenado. En este caso usaremos el modelo Nano de Yolo para ver el comportamiento en el proceso de entrenamiento y posteriormente aumentar el tamaño del modelo a entrenar.

In [7]:
from ultralytics import YOLO
import cv2 as cv2
from PIL import Image

# Load a pretrained YOLO model (recommended for training)
model = YOLO('yolov8n.pt')

100%|██████████| 6.23M/6.23M [00:00<00:00, 118MB/s]


In [8]:
import os

# Función para corregir las anotaciones
def fix_annotations(annotation_folder):
    for root, dirs, files in os.walk(annotation_folder):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                fix_annotation_file(file_path)

# Función para corregir un archivo de anotación específico
def fix_annotation_file(file_path):
    with open(file_path, "r+") as file:
        lines = file.readlines()
        file.seek(0)
        for line in lines:
            parts = line.strip().split()
            if len(parts) > 5:
                parts = parts[:5]  # Mantener solo las primeras 5 columnas
            fixed_line = " ".join(parts) + "\n"
            file.write(fixed_line)
        file.truncate()

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/val"

# Corregir las anotaciones
fix_annotations(annotation_folder)

# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/test"

# Corregir las anotaciones
fix_annotations(annotation_folder)
# Directorio donde se encuentran los archivos de anotaciones
annotation_folder = "dataSet/labels/train"

# Corregir las anotaciones
fix_annotations(annotation_folder)


In [9]:
# Train the model using the 'coco8.yaml' dataset for 3 epochs
results = model.train(data='dataSet.yaml',
                      epochs=100,
                      imgsz=1200,
                      plots = True)

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=dataSet.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=1200, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

100%|██████████| 755k/755k [00:00<00:00, 28.1MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/dataSet/labels/train... 1408 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1408/1408 [00:01<00:00, 1311.40it/s]

train: New cache created: /content/dataSet/labels/train.cache


...


100 epochs completed in 0.977 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.3MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):  25%|██▌       | 2/8 [00:00<00:02,  2.35it/s]

WARNING ⚠️ NMS time limit 3.600s exceeded


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:26<00:00,  3.35s/it]


                   all        228       8684      0.874      0.898      0.934      0.658
Speed: 0.4ms preprocess, 2.7ms inference, 0.0ms loss, 18.3ms postprocess per image
Results saved to runs/detect/train


In [10]:
metrics = model.val(split = 'test', plots = True)

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (NVIDIA L4, 22700MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/dataSet/labels/test... 177 images, 0 backgrounds, 0 corrupt: 100%|██████████| 177/177 [00:00<00:00, 1311.40it/s]

val: New cache created: /content/dataSet/labels/test.cache



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 12/12 [00:15<00:00,  1.31s/it]


                   all        177       2773      0.938      0.881      0.952      0.701
Speed: 0.9ms preprocess, 25.1ms inference, 0.0ms loss, 25.1ms postprocess per image
Results saved to runs/detect/train2


In [12]:
metrics.results_dict

{'metrics/precision(B)': 0.9377022998183243,
 'metrics/recall(B)': 0.8813559322033898,
 'metrics/mAP50(B)': 0.9519925065794119,
 'metrics/mAP50-95(B)': 0.7010531890269575,
 'fitness': 0.7261471207822029}

In [13]:
#model.predict(source = 'dataSet/images/test/image_0.jpg', save = True)
results = model.track(source="test.avi", show=True)

requirements: Ultralytics requirement ['lapx>=0.5.2'] not found, attempting AutoUpdate...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 12.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 6.0s, installed 1 package: ['lapx>=0.5.2']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 4 Mov

In [18]:
%%capture
results = model('test.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/test.avi: 768x1216 4 Movils, 10.3ms
video 1/1 (frame 2/60) /content/test.avi: 768x1216 4 Movils, 9.5ms
video 1/1 (frame 3/60) /content/test.avi: 768x1216 5 Movils, 10.0ms
video 1/1 (frame 4/60) /content/test.avi: 768x1216 5 Movils, 9.9ms
video 1/1 (frame 5/60) /content/test.avi: 768x1216 5 Movils, 10.3ms
video 1/1 (frame 6/60) /content/test.avi: 768x1216 5 Movils, 9.7ms
video 1/1 (frame 7/60) /content/test.avi: 768x1216 

In [25]:
%%capture
results = model('Suppl_1.avi', save=True)



WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/60) /content/Suppl_1.avi: 768x1216 31 Movils, 12.1ms
video 1/1 (frame 2/60) /content/Suppl_1.avi: 768x1216 31 Movils, 10.3ms
video 1/1 (frame 3/60) /content/Suppl_1.avi: 768x1216 30 Movils, 10.0ms
video 1/1 (frame 4/60) /content/Suppl_1.avi: 768x1216 22 Movils, 11.0ms
Speed: 7.1ms preprocess, 10.8ms inference, 1.8ms postprocess per image at shape (1, 3, 768, 1216)
Results saved to runs/detect/train4


In [21]:
# Export the model
model.export()

Ultralytics YOLOv8.2.5 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'runs/detect/train/weights/best.pt' with input shape (1, 3, 1216, 1216) BCHW and output shape(s) (1, 5, 30324) (6.0 MB)

TorchScript: starting export with torch 2.2.1+cu121...
TorchScript: export success ✅ 1.7s, saved as 'runs/detect/train/weights/best.torchscript' (12.2 MB)

Export complete (3.1s)
Results saved to /content/runs/detect/train/weights
Predict:         yolo predict task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216  
Validate:        yolo val task=detect model=runs/detect/train/weights/best.torchscript imgsz=1216 data=dataSet.yaml  
Visualize:       https://netron.app


'runs/detect/train/weights/best.torchscript'

In [26]:
import shutil
archivo = shutil.make_archive('ModelInformation', 'zip','/content/runs')